##### Token Initialization

In [1]:
import pandas as pd
df=pd.read_csv('PoetryFoundationData.csv')
df.columns

col1=df['Poem']
entry=""
for i in range(0,21):
    for j in col1[i]:
        entry=entry+j

import re
text=re.split(r'(\s)', entry)
#Removing spaces

new_list=[]
for i in text:
    new=i.strip()
    if new!="":
        new_list.append(new)

#print(new_list)
new_list.sort()

##### Token ID

In [2]:
id={}
for i in range(len(new_list)):
    id.update({new_list[i]:i})

print(id)

{'"smile."': 0, '"whadda': 1, '(I': 2, '(it': 3, '(yellow': 4, '.': 7, '100s': 8, '20,': 10, '5': 12, 'A': 13, 'Act': 14, 'Adcock': 15, 'After': 16, 'All': 17, 'America': 18, 'And': 26, 'Are': 27, 'As': 30, 'Backward': 31, 'Beating': 32, 'Beatles': 33, 'Beauty': 34, 'Behind': 35, 'Blinds': 36, 'Book': 37, 'Both': 38, 'Boys': 40, 'Bungee': 41, 'Bunnies': 42, 'But': 47, 'Can': 48, 'Chevrolet': 49, 'Chevy': 50, 'Corner,': 51, 'Danny,': 52, 'Day': 53, 'Destined': 54, 'Dictated': 55, 'Did': 57, 'Dog': 58, 'Don’t': 59, 'Duke,': 60, 'Elm': 61, "Elmer's": 62, 'Everything': 64, 'Farm?': 65, 'Feather': 66, 'Flick.': 67, 'For': 69, 'Forward': 70, 'From': 71, 'God': 72, 'Gone': 73, 'Grove,': 74, 'Hall,': 75, 'He': 84, 'Her': 86, 'Hides': 87, 'His': 89, 'Hour': 90, 'House,': 91, 'How': 92, 'Hydrangea.': 93, 'I': 178, "I'm": 179, "I've": 181, 'I.': 183, 'II': 184, 'If': 188, 'In': 192, 'Invisible': 193, 'Is': 195, 'It': 197, "It's": 199, 'It’s': 201, 'I’d': 202, 'I’m': 204, 'Jack': 208, 'Jack.': 209

##### Tokenizer

In [3]:
class tokenizer:
    def __init__(self, dic):
        self.dic=dic

        dic1={}
        for i,j in dic.items():
            dic1.update({j:i})

        self.inverse_dic=dic1
    
    def encode(self, txt):
        text=re.split(r'([,:?;]|\s)', txt)
        id=[]
        for i in text:
            j=i.strip()
            if j!="":
                id.append(self.dic[j])
        
        return id
    
    def decode(self, id):
        st=""
        for i in range(len(id)):
            if id==len(id)-1:
                st=st+self.inverse_dic[id[i]]
            else:
                st=st+self.inverse_dic[id[i]]+" "
        
        return st

token1=tokenizer(id)
entry=token1.encode("It's a big Dog")
final=token1.decode(entry)
print(final)


It's a big Dog 


##### Byte Pair Encoding

In [4]:
import tiktoken
tokenizer1=tiktoken.get_encoding("gpt2")
codes=tokenizer1.encode("I love BMW")
text=tokenizer1.decode(codes)
print(text)

I love BMW


##### Sliding Window Algorithm

In [5]:
text_to_encode=col1[5]
limit=32
actual=text_to_encode[0:limit]

X=[]
Y=[]
for i in actual.split(" "):
    Y.append(i.strip())
    X.append(i.strip())


Y.remove(Y[0])
X.remove(X[-1])

print(actual+"\n")

X_id=[]
Y_id=[]
for i in X:
    X_id.append(id[i])

for i in Y:
    Y_id.append(id[i])

import torch

x_tensor=[]
y_tensor=[]

for i in range(0, len(X_id), 2):
    li=[]
    li.append(X_id[i])
    li.append(X_id[i+1])

    li1=[]
    li1.append(Y_id[i])
    li1.append(Y_id[i+1])

    x_tensor.append(li)
    y_tensor.append(li1)

x_tensor=torch.tensor(x_tensor)
y_tensor=torch.tensor(y_tensor)
print(x_tensor)
print(y_tensor)


You are the start of the week

tensor([[ 342,  582],
        [2694, 2396],
        [1956, 2694]])
tensor([[ 582, 2694],
        [2396, 1956],
        [2694, 2975]])


##### Embedding

In [13]:
embedding=torch.nn.Embedding(len(new_list), 3)
#print(embedding.weight)
embebbed_layer=embedding(x_tensor)
print(embebbed_layer)

tensor([[[ 1.3269,  0.4194, -0.4350],
         [-0.4237, -0.2887,  1.0424]],

        [[ 0.8028, -1.1081,  0.0571],
         [-0.0893, -0.0793, -0.8654]],

        [[ 0.0939, -1.7022, -0.1036],
         [ 0.8028, -1.1081,  0.0571]]], grad_fn=<EmbeddingBackward0>)


##### Attention Scores

In [28]:
new_dim=[]
for i in embebbed_layer:
    for j in i:
        new_dim.append(j.tolist())   

new_dim=torch.tensor(new_dim)

attention=torch.empty(6,6)


for i,j in enumerate(new_dim):
    for k,l in enumerate(new_dim):
        attention[i,k]=torch.dot(j,l)

attention=torch.softmax(attention, dim=1)
print(attention)

context=attention@new_dim
print("\n--------------Context Vector---------------\n")
print(context)

tensor([[0.5948, 0.0228, 0.1262, 0.0889, 0.0412, 0.1262],
        [0.0396, 0.4761, 0.1284, 0.0532, 0.1741, 0.1284],
        [0.0744, 0.0435, 0.2730, 0.0405, 0.2957, 0.2730],
        [0.1787, 0.0616, 0.1381, 0.3062, 0.1772, 0.1381],
        [0.0162, 0.0393, 0.1971, 0.0346, 0.5156, 0.1971],
        [0.0744, 0.0435, 0.2730, 0.0405, 0.2957, 0.2730]])

--------------Context Vector---------------

tensor([[ 0.9782, -0.1140, -0.3018],
        [ 0.0686, -0.7061,  0.4296],
        [ 0.5427, -1.0929, -0.0215],
        [ 0.4221, -0.5749, -0.2812],
        [ 0.3667, -1.3219, -0.0269],
        [ 0.5427, -1.0929, -0.0215]])
